### Graded Lab 4

Hello ! Welcome to Graded Lab of Module 4.

In the last assignment we had worked on correlation , hypothesis which business wanted us to test.
Its high time when we look at the missing values & outliers stuff.
**Here we will work on missing values & outliers.**

In case if you are not able to recollect the problem description and data description then mentioning it below.

Lets look at the problem statement,

*Client: ABC Retail, Incorporated, rest-of-the-world division* 

***Project name: Online retail sales analysis*** 

An online retailer, ABC, Inc., operates in nearly 100 countries worldwide, selling furniture, office supplies and technology products to customers in three segments: consumer, corporate and home office. ABC, Inc. is a US-based company, and it has two major divisions: US and rest of the world. We are working with the rest of the world division of the company. 

They have provided us with online sales transaction data from 2011 to 2014.

We are given 3 datasets:-

1. Data on each sale; 51290 records; all data in US dollars
It contains fields like
**order_id** (identifier) ,order_date ,ship_date ,ship_mode ,**customer_id**(identifier) ,product_id ,category ,sub_category ,product_name ,sales ,quantity ,discount ,profit ,shipping_cost ,order_priority ,**vendor_code** (identifier) 


2. Data on the customers; 1590 records 
It contains fields like
**customer_id** (identifier) ,customer_name ,city ,state ,country ,postal_code ,segment ,market ,region 

3. Data on vendors who supply the retailer; 65 records 
It contains fields like
vendor ,**vendor_code** (identifier) 

We need to analyze the data and need to provide answer to different questions asked by company officials.

# Instructor update: Jan 29, 2024

In [ ]:
# importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from itertools import combinations
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

### Reading sales data
sales = pd.read_csv('sales_data_M4.csv')

### Reading customer data
cust = pd.read_csv(r'customers.csv',encoding='iso-8859-1')

### Reading vendor data
vend = pd.read_csv(r'vendors.csv')

sales.head()

In order to solve the next questions , we need to combine all the 3 datasets into a single dataframe such that every details of sales dataframe are intact. So here we have written a data processing function.
There are 2 tasks which are to be performed.
1. Merge/ Join all the 3 datasets into a single dataframe such that every details of sales dataframe are intact. (Understand which should be the joining key , type of join , refer .merge() function of pandas)
2. Convert 'order_date' into a datetime column.
**Return output as a dataframe**

In [ ]:
#### data_merging & order_date processing , data1 will be sales , data2 will be customer dataset & data3 will be vendor dataset.

def data_process(data1,data2,data3):
    # your code here
    return data

In [ ]:
sales= data_process(data1=sales.copy(),data2=cust.copy(),data3=vend.copy())

In [ ]:
assert sales['order_date'].dtypes=='<M8[ns]' ,'Make sure if you have converted order_date into a datetime format correctly or not.'
assert sales.shape== (51290,26) ,'Checking size and shape of dataframe after merging is a very important check.'

Note that in this dataset we are purposefully introducing some of the missing values.

### Q.1 Return column names & missing values percentage for the columns which has missing value% >50. Output should be a dictionary. 

For eg:- { column_name : value%} i.e {'A':54.50,'B':59.75}, make sure to round off percentages to 2 decimals.

In [ ]:
def missing_value_col(data):
    # your code here

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.

#### **Senior data scientists after consulting with the business have decided that such columns needs to be dropped before going further.**

In [ ]:
high_missing_col=list(missing_value_col(data=sales).keys()) #Enter the columns with high % of missing values
### We will drop columns with such high missing values
sales.drop(high_missing_col,axis=1,inplace=True)

### Q.2 Fill the missing data in 'category' column with appropriate method.  
(Hint:- Remember variable 'category' & 'sub-category' are related & hierachy goes like this category -> sub-category)

Approach:-

1. Create a key-pair dictionary where key will be product category & pair values will be sub-categories.

2. Wherever category value is missing check for corresponding sub-category value. 

3. From sub-category value trace back its original product category value by looking in dictionary, get a product value.

4. Fill the blank product category value by the same.

In [ ]:
### create a dictionary where keys will be categories & sub-categories as values. store in inside Category_dict.

def cat_dict(data,main_col,sub_col):    
    # your code here
    return Category_dict 

def category_subcategory_map_dataframe(df, Category_dict, main_col, sub_col):
    def category_subcategory_map(row):
        if pd.isna(row[main_col]):  # If the main column is null
            if not pd.isna(row[sub_col]):  # If the sub column is not null
                for k, v in Category_dict.items():
                    if row[sub_col] in Category_dict[k]:
                        return k  # k is the correct sub-category for the category value v
        else:
            return row[main_col]  # If the main column is not null, return its existing value

    df[main_col] = df.apply(category_subcategory_map, axis=1)
    return df[main_col]

In [ ]:
Category_dict=cat_dict(data=sales,main_col='category',sub_col='sub_category')
sales['category']=category_subcategory_map_dataframe(df=sales, Category_dict=Category_dict, main_col='category', sub_col='sub_category')

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.

### Q.3 ) Fill the missing values in 'sales' column by average value of that column. Return output series.

In [ ]:
def fill_missing(data,col):
    # your code here

In [ ]:
sales['sales']=fill_missing(data=sales,col='sales')

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.
assert sales['sales'].isna().sum()==0 ,'Missing values found for sales.'

### Q.4 ) Discretize the 'sales' column wrto following groups. Each group should contain number of orders as values.
Groups :- [0,12),[12,37),[37,113),[113,186),[186,1200).

**Note :-[a,b) indicates that group includes a but excudes b.**

**Return output series with group name as index & number of orders as values.**

For eg:-

[186, 1200)             123

[37, 113)               123

[12, 37)                123

[113, 186)              123

[0, 12)                 123

Name: sales, dtype: int64

In [ ]:
def manual_bin(data,col):
    # your code here

In [ ]:
assert type(manual_bin(data=sales,col='sales'))==pd.Series, "Please provide output in series format."

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.
assert manual_bin(data=sales,col='sales').values[0]==18550, 'Make sure you have returned correct series values.'

### Q.5 )  Sometimes, it is difficult to comprehend the quantity sold ('quantity') value as a numeric value. So, the business team wants to observe the sales as High, Medium and Low categories.  

### Write a code to discretize the sales data and  return an output series containing High, Medium and Low categories as index & the respective number of records as values.

Hint : Here the bounds for High , Medium , Low categories are not given hence we can consider the equal split of data for defining the bounds , think of using pd.cut() function.

In [ ]:
def discrete_bin(data,col):
    # your code here

In [ ]:
assert type(discrete_bin(data=sales,col='quantity'))==pd.Series, "Please provide output in series format."

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.

### Q.6. In order to perform further analysis, we intend to utilize the 'ship_mode' variable, which has categorical values ('First Class'(0) > 'Second Class'(1) > 'Standard Class'(2) > 'Same Day'(3)). To facilitate this analysis, we need to convert 'ship_mode' into a numerical feature. Please encode 'ship_mode' into a numerical format.
### Return output series.

Hint : Think of using the OrdinalEncoder() function from scikit-learn.

In [ ]:
## Select Appropriate priority
priority=None # Write your code in place of None 
def transforming_ordered_var(data,col,priority,nan_val):
    #data: dataset to be used
    #col : column name which is to be encoded.
    #priority: list of categories sorted from highest order of importance to lowest order importance.
    #nan_val : it will be a value assigned in place of nan's.   
    # your code here

In [ ]:
sales['shipping_mode_encoded']=transforming_ordered_var(data=sales,col='ship_mode',priority=priority,nan_val=5)

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.
assert int(transforming_ordered_var(data=sales,col='ship_mode',priority=priority,nan_val=5).value_counts().loc[lambda x:x.index==5])==1570

### Q.7) Outliers can be defined in different ways. For each of the following definitions of outliers, what percent of the values in the  'sales' are considered outliers? Give the answer as a percent rounded upto two decimal places.


Note -
1. 'IQR_detection' : Less than (Q1 - 1.5*IQR) and greater than (Q3 + 1.5*IQR) are considered outliers

2. 'percentile_detection': Below the 3rd percentile and above the 97th percentile are considered outliers

3. 'mean_SD' : Less than (mean - 3*SD) and greater than (mean + 3*SD) are considered outliers 

In [ ]:
def detect_outliers(data,col,method):
    if method=='IQR_detection':
        # your code here
    elif method=='percentile_detection':
        # your code here
    elif method=='mean_SD':
        # your code here
    return outliers_perc

In [ ]:
assert type(detect_outliers(data=sales,col='sales',method='IQR_detection'))==np.float, 'Make sure that output is returned as float'
assert detect_outliers(data=sales,col='sales',method='IQR_detection')==10.06, 'Make sure that output is rounded upto 2 decimals'

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.

### Q.8) Detect upper end outliers in profit column by 'IQR_detection'. Return number of upper end outliers.

In [ ]:
def detect_upperend_outliers(data,col):
    # your code here
    return outliers.shape[0] ##data frame named 'outliers' which contains only outliers.

In [ ]:
assert type(detect_upperend_outliers(data=sales,col='profit'))==int , 'No of outliers should be an integer , make sure that you are returning output in integer format.'

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.

### Q.9) Cap & Floor outliers at 97% at 3%. Return the processed column as output.

In [ ]:
def cap_and_floor_column(column, cap_percentile, floor_percentile):
    # your code here
    return capped_and_floored_column ## Capped ,Floored series

In [ ]:
sales['profit_outlier_capped'] = cap_and_floor_column(column=sales['profit'], cap_percentile=97, floor_percentile=3)

In [ ]:
assert round(np.quantile(sales['profit_outlier_capped'],0.97),4)==301.9693, 'Make sure that you are flooring and capping with correct percentiles & returning the correct output.'
assert round(np.quantile(sales['profit_outlier_capped'],0.03),4)==-144.632, 'Make sure that you are flooring and capping with correct percentiles & returning the correct output.'

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.

### Q.10) Business team is interested in knowing how many vendors have profit_outlier_capped <10000 & vendors with profit_outlier_capped >10000. Return the dataframe with 3 columns vendor  , profit_outlier_capped , profit_bins & sort it with 'profit_outlier_capped' in descending order.

Instructions:-
1. Bins should be in (a,b] format. (a,b] indicates that bin excludes a but incudes b.
2. Make sure that for **1st interval only** it should be **left inclusive** i.e [a,b].
3. Segment the ''profit_outlier_capped'' variable in 2 bins. (binendpoints should be min value of profit<10000,10000,max value of profit>10000).

In [ ]:
def profit_vendor(data,col):
    # your code here

In [ ]:
assert type(profit_vendor(data=sales,col='profit_outlier_capped'))==pd.DataFrame ,'Make sure that your output is a dataframe '
assert str(profit_vendor(data=sales,col='profit_outlier_capped')['profit_bins'].unique()[0])=='(10000.0, 76469.853]' , 'Make sure that you are selecting bin cut-points(a & b) properly'
assert str(profit_vendor(data=sales,col='profit_outlier_capped')['profit_bins'].unique()[1])=='(2732.9049999999997, 10000.0]', 'Make sure that you are selecting bin cut-points(a & b) properly'

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.